In [1]:
import pandas as pd
df = pd.read_csv('/content/daum_movie_review.csv')
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [2]:
df.title.value_counts()

신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: title, dtype: int64

In [3]:
from sklearn.model_selection import train_test_split

# 데이터와 라벨을 학습 세트와 평가 세트로 분리
# 비율을 지정하지 않으면 75:25로 분할됨
X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=0)

print('#Train set size:', len(X_train)) # 실제로 몇 개의 특성이 사용됐는지 확인
print('#Test set size:', len(X_test))

#Train set size: 11043
#Test set size: 3682


In [5]:
!pip3 install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 37.6 MB/s eta 0:00:00


In [6]:
from konlpy.tag import Okt
okt = Okt()

print(okt.morphs(X_train[1])) # 둘째 리뷰에 대해 형태소 단위로 tokenize
print(okt.nouns(X_train[1])) # 둘째 리뷰에서 명사만 추출

['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

# logistic regression 분류기 선언
# 충분한 학습을 위해 max_iter를 1,000으로 설정, 기본은 100
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train) # 분류기 학습
# train data 예측 정확도
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
# test data 예측 정확도
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.756
#Test set score: 0.694


In [8]:
print('실제영화제목, 예측한 제목, 리뷰')
for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
  print(content)

실제영화제목, 예측한 제목, 리뷰
('범죄도시', '신과함께', '오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.')
('범죄도시', '범죄도시', '조연들이 눈에 박힌다. 간만에 집중 ㅎ')
('코코', '코코', '대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??')
('신과함께', '신과함께', '돈이 안아까웠던 영화ᆞᆞ  정말 좋았다')
('신과함께', '신과함께', '역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.')
('택시운전사', '택시운전사', '민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.')
('신과함께', '신과함께', '잠만 자다 왔음')
('신과함께', '신과함께', '오랜만에 잼있고 좋은 영화를 봤다')
('범죄도시', '신과함께', '잼남')
('범죄도시', '인피니티 워', '대박~~')


In [9]:
# 명사 대신 모든 형태소를 사용
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# 충분한 학습을 위해 max_iter를 1,000으로 설정, 기본은 100
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# train data 예측정확도
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
# test data 예측정확도
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.777
#Test set score: 0.695


In [21]:
def twit_tokenizer(text): # 전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
  target_tags = ['Noun', 'Verb', 'Adjective']
  result = []
  for word, tag in okt.pos(text, norm=True, stem=True):
   if tag in target_tags:
     result.append(word)

  return result

# 명사, 동사, 형용사를 이용해 tfidf 생성
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.784
#Test set score: 0.712


In [22]:
# 모든 형태소를 다 사용하고 품사를 알 수 있게 한다면?
def twit_tokenizer2(text):
  result = []
  for word, tag in okt.pos(text, norm=True, stem=True):
    result.append('/'.join([word, tag])) # 단어의 품사를 구분할 수 있게 함
  
  return result

print(twit_tokenizer2(X_train[1]))

['몰입/Noun', '하다/Verb', '없다/Adjective', './Punctuation', '어렵다/Adjective', '생각/Noun', '하다/Verb', '필요없다/Adjective', './Punctuation', '내/Noun', '가/Josa', '전투/Noun', '에/Josa', '참여/Noun', '한/Determiner', '듯/Noun', '손/Noun', '에/Josa', '땀/Noun', '이남/Noun', './Punctuation']


In [23]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.789
#Test set score: 0.718


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(
    X_train_tfidf, y_train, test_size=0.2, random_state=42)

max_score = 0
max_alpha = 0
for alpha in np.arange(0.1, 10, 0.1): # alpha를 0.1부터 10까지 0.1씩 증가
  ridge_clf = RidgeClassifier(alpha=alpha) # 릿지 분류기 선언
  ridge_clf.fit(X_train_ridge, y_train_ridge) # 학습
  # 검정 데이터셋에 대해 정확도를 측정
  score = ridge_clf.score(X_val_ridge, y_val_ridge)
  if score > max_score: # 정확도가 이전의 정확도 최댓값보다 크면 최댓값을 변경한다.
    max_score = score
    max_alpha = alpha
print('#Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

#Max alpha 1.600 at max validation score 0.727


In [26]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=1.6)
ridge_clf.fit(X_train_tfidf, y_train)
print('#Ridge Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('#Ridge Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=0.5)
lasso_clf.fit(X_train_tfidf, y_train)
print('#Lasso Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('#Lasso Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print(
    '#Used features count: {}'.format(np.sum(lasso_clf.coef_ !=0)),
    'out of',
    X_train_tfidf.shape[1]
)

#Ridge Train set score: 0.807
#Ridge Test set score: 0.726
#Lasso Train set score: 0.703
#Lasso Test set score: 0.696
#Used features count: 957 out of 2000


In [27]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB(alpha=0.1)
NB_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.768
Test set score: 0.704
